In [1]:
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo 

# fetch dataset 
chess_king_rook_vs_king_pawn = fetch_ucirepo(id=22) 
  
# data (as pandas dataframes) 
X = chess_king_rook_vs_king_pawn.data.features 
y = chess_king_rook_vs_king_pawn.data.targets   
  
# Combine X and y into a single DataFrame
df = pd.concat([X, y], axis=1)

# Rename the target column to 'Y'
df = df.rename(columns={df.columns[-1]: 'Y'})
df = df.dropna()

In [2]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# Assuming df is your initial DataFrame
# Split data
X = df.drop('Y', axis=1)
y = df['Y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Dictionary of regression models with their parameter grids
models = {
    'Linear Regression': (LinearRegression(), {}),
    'Ridge Regression': (Ridge(), {'alpha': [0.1, 1.0, 10.0]}),
    'Lasso Regression': (Lasso(), {'alpha': [0.1, 1.0, 10.0]}),
    'ElasticNet': (ElasticNet(), {'alpha': [0.1, 1.0, 10.0], 'l1_ratio': [0.2, 0.5, 0.8]}),
    'Decision Tree': (DecisionTreeRegressor(), {'max_depth': [3, 5, 7]}),
    'Random Forest': (RandomForestRegressor(), {'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 7]}),
    'Gradient Boosting': (GradientBoostingRegressor(), {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.3]}),
    'XGBoost': (XGBRegressor(), {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.3]}),
    'LightGBM': (LGBMRegressor(), {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.3]}),
    'CatBoost': (CatBoostRegressor(verbose=0), {'iterations': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.3]})
}

# Dictionary to store results
results = {}

for name, (model, param_grid) in models.items():
    start_time = time.time()
    
    # Perform GridSearchCV
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train_scaled, y_train)
    
    # Get the best model
    best_model = grid_search.best_estimator_
    
    # Training time for best parameters
    training_start = time.time()
    best_model.fit(X_train_scaled, y_train)
    training_time = time.time() - training_start
    
    # Inference time for best parameters
    inference_start = time.time()
    y_pred = best_model.predict(X_test_scaled)
    inference_time = time.time() - inference_start
    
    # Compute metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    # Computation time (total run time)
    computation_time = time.time() - start_time
    
    results[name] = {
        'RMSE': rmse,
        'R-squared': r2,
        'Training Time': training_time,
        'Inference Time': inference_time,
        'Computation Time': computation_time,
        'Best Parameters': grid_search.best_params_
    }

# Convert results to a DataFrame
result = pd.DataFrame(results).T  # Transpose to have models as rows

# Display results
print(result)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 24, number of used features: 0
[LightGBM] [Info] Start training from score -25.224351
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the sp

In [14]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor

# Start timing the entire process
start_time = time.time()

# Assuming df is your initial DataFrame
# Split data
X = df.drop('Y', axis=1)
y = df['Y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the MLP model and parameter grid
mlp = MLPRegressor(random_state=42, max_iter=1000)
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive'],
}

# Perform GridSearchCV
grid_search = GridSearchCV(mlp, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Training time for best parameters
training_start = time.time()
best_model.fit(X_train_scaled, y_train)
training_time = time.time() - training_start

# Inference time for best parameters
inference_start = time.time()
y_pred = best_model.predict(X_test_scaled)
inference_time = time.time() - inference_start

# Compute metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Calculate total computation time
computation_time = time.time() - start_time

result.loc['MLP'] = [rmse, r2, training_time, inference_time, computation_time, grid_search.best_params_]

# Display results
print(result)

# Print the best parameters
print("\nBest Parameters:")
print(grid_search.best_params_)


c:\Users\Tommy\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


                         RMSE R-squared Training Time Inference Time  \
Linear Regression    0.211628  0.999992      0.001003            0.0   
Ridge Regression     0.905454  0.999858      0.000999            0.0   
Lasso Regression     0.383952  0.999974           0.0            0.0   
ElasticNet           4.404609  0.996642      0.001995            0.0   
Decision Tree       36.807797    0.7655      0.000998            0.0   
Random Forest       37.345571  0.758598      0.391953       0.023935   
Gradient Boosting   32.551013  0.816603      0.060839       0.000996   
XGBoost             47.556448  0.608545      0.066822       0.000997   
LightGBM           105.104923 -0.912095      0.005985       0.000997   
CatBoost             61.82527    0.3384       0.16257       0.001988   
MLP                  7.094997  0.991287      0.594412       0.000998   

                  Computation Time  \
Linear Regression         0.055115   
Ridge Regression          0.050876   
Lasso Regression     

c:\Users\Tommy\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [15]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.base import BaseEstimator, RegressorMixin

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is your initial DataFrame
# Split data
X = df.drop('Y', axis=1)
y = df['Y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.FloatTensor(y_train.values).unsqueeze(1).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.FloatTensor(y_test.values).unsqueeze(1).to(device)

class DNN(nn.Module):
    def __init__(self, input_dim, hidden_dims):
        super(DNN, self).__init__()
        layers = []
        prev_dim = input_dim
        for dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, dim))
            layers.append(nn.ReLU())
            prev_dim = dim
        layers.append(nn.Linear(prev_dim, 1))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

class DNNRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, input_dim, hidden_dims, learning_rate=0.001, batch_size=32, num_epochs=100):
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.model = DNN(input_dim, hidden_dims).to(device)
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

    def fit(self, X, y):
        train_dataset = TensorDataset(torch.FloatTensor(X).to(device), torch.FloatTensor(y).unsqueeze(1).to(device))
        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)

        self.model.train()
        for epoch in range(self.num_epochs):
            for batch_X, batch_y in train_loader:
                self.optimizer.zero_grad()
                outputs = self.model(batch_X)
                loss = self.criterion(outputs, batch_y)
                loss.backward()
                self.optimizer.step()
        return self

    def predict(self, X):
        self.model.eval()
        with torch.no_grad():
            X_tensor = torch.FloatTensor(X).to(device)
            predictions = self.model(X_tensor)
        return predictions.cpu().numpy()

# Define the parameter grid for GridSearchCV
param_grid = {
    'hidden_dims': [[50], [100], [50, 50], [100, 50]],
    'learning_rate': [0.0001, 0.001, 0.01],
    'batch_size': [32, 64, 128],
    'num_epochs': [50, 100]
}

# Create the DNNRegressor instance
dnn_regressor = DNNRegressor(input_dim=X_train.shape[1], hidden_dims=[50])

# Perform GridSearchCV
grid_search = GridSearchCV(dnn_regressor, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Training time for best parameters
training_start = time.time()
best_model.fit(X_train_scaled, y_train)
training_time = time.time() - training_start

# Inference time for best parameters
inference_start = time.time()
y_pred = best_model.predict(X_test_scaled)
inference_time = time.time() - inference_start

# Compute metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Calculate total computation time
computation_time = time.time() - start_time

# Assuming result is your existing DataFrame
# Store results in the existing result DataFrame
result.loc['DNN'] = [rmse, r2, training_time, inference_time, computation_time, grid_search.best_params_]

print(result)

# Print the best parameters
print("\nBest Parameters:")
print(grid_search.best_params_)


Using device: cpu
                         RMSE R-squared Training Time Inference Time  \
Linear Regression    0.211628  0.999992      0.001003            0.0   
Ridge Regression     0.905454  0.999858      0.000999            0.0   
Lasso Regression     0.383952  0.999974           0.0            0.0   
ElasticNet           4.404609  0.996642      0.001995            0.0   
Decision Tree       36.807797    0.7655      0.000998            0.0   
Random Forest       37.345571  0.758598      0.391953       0.023935   
Gradient Boosting   32.551013  0.816603      0.060839       0.000996   
XGBoost             47.556448  0.608545      0.066822       0.000997   
LightGBM           105.104923 -0.912095      0.005985       0.000997   
CatBoost             61.82527    0.3384       0.16257       0.001988   
MLP                  7.094997  0.991287      0.594412       0.000998   
DNN                109.746297 -1.084697      0.063799            0.0   

                  Computation Time  \
Linear 

c:\Users\Tommy\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
72 fits failed out of a total of 216.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
72 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Tommy\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\3484552703.py", line 64, in fit
ValueError: could not determine the shape of object type 'Series'

  warnings.warn(some_fits_failed_message, FitFailedWarning)
c:\Users\Tommy\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:1102: UserWarni

In [16]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is your initial DataFrame
# Split data
X = df.drop('Y', axis=1)
y = df['Y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.FloatTensor(y_train.values).unsqueeze(1).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.FloatTensor(y_test.values).unsqueeze(1).to(device)

class CrossLayer(nn.Module):
    def __init__(self, input_dim):
        super(CrossLayer, self).__init__()
        self.weight = nn.Parameter(torch.Tensor(input_dim, 1))
        self.bias = nn.Parameter(torch.Tensor(input_dim, 1))
        nn.init.xavier_uniform_(self.weight)
        nn.init.zeros_(self.bias)

    def forward(self, x0, x):
        x = x.unsqueeze(2)
        x0 = x0.unsqueeze(2)
        interaction = torch.matmul(x0, torch.matmul(x.transpose(1, 2), self.weight))
        return x0.squeeze(2) + interaction.squeeze(2) + self.bias.T

class DCN(nn.Module):
    def __init__(self, input_dim, cross_layers, deep_layers):
        super(DCN, self).__init__()
        self.cross_layers = nn.ModuleList([CrossLayer(input_dim) for _ in range(cross_layers)])
        
        deep_dims = [input_dim] + deep_layers
        self.deep_layers = nn.ModuleList([nn.Linear(deep_dims[i], deep_dims[i+1]) for i in range(len(deep_layers))])
        self.deep_activation = nn.ReLU()
        
        self.final_layer = nn.Linear(deep_layers[-1] + input_dim, 1)

    def forward(self, x):
        x0 = x
        cross_out = x
        for layer in self.cross_layers:
            cross_out = layer(x0, cross_out)
        
        deep_out = x
        for layer in self.deep_layers:
            deep_out = self.deep_activation(layer(deep_out))
        
        combined = torch.cat([cross_out, deep_out], dim=1)
        return self.final_layer(combined)

def objective(trial):
    # Define hyperparameters to tune
    cross_layers = trial.suggest_int('cross_layers', 1, 5)
    deep_layers = [trial.suggest_int(f'deep_layer_{i}', 32, 256) for i in range(3)]
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the DCN model
    model = DCN(X_train.shape[1], cross_layers, deep_layers).to(device)

    # Define loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test_tensor)
        mse = mean_squared_error(y_test_tensor.cpu().numpy(), y_pred.cpu().numpy())
        rmse = np.sqrt(mse)

    return rmse

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final DCN model with the best hyperparameters
best_model = DCN(X_train.shape[1], 
                 best_params['cross_layers'], 
                 [best_params[f'deep_layer_{i}'] for i in range(3)]).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    y_pred = best_model(X_test_tensor)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_true = y_test_tensor.cpu().numpy()
    y_pred = y_pred.cpu().numpy()
    
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['DCN'] = [rmse, r2, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")

[I 2024-07-26 08:08:56,710] A new study created in memory with name: no-name-182fff70-c98f-4851-b413-5a94b6c913be


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\95374287.py:79: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 08:08:57,040] Trial 0 finished with value: 9.475296020507812 and parameters: {'cross_layers': 4, 'deep_layer_0': 225, 'deep_layer_1': 76, 'deep_layer_2': 127, 'learning_rate': 0.03614504961249343, 'batch_size': 64, 'num_epochs': 42}. Best is trial 0 with value: 9.475296020507812.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\95374287.py:79: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 08:08:5

                         RMSE R-squared Training Time Inference Time  \
Linear Regression    0.211628  0.999992      0.001003            0.0   
Ridge Regression     0.905454  0.999858      0.000999            0.0   
Lasso Regression     0.383952  0.999974           0.0            0.0   
ElasticNet           4.404609  0.996642      0.001995            0.0   
Decision Tree       36.807797    0.7655      0.000998            0.0   
Random Forest       37.345571  0.758598      0.391953       0.023935   
Gradient Boosting   32.551013  0.816603      0.060839       0.000996   
XGBoost             47.556448  0.608545      0.066822       0.000997   
LightGBM           105.104923 -0.912095      0.005985       0.000997   
CatBoost             61.82527    0.3384       0.16257       0.001988   
MLP                  7.094997  0.991287      0.594412       0.000998   
DNN                109.746297 -1.084697      0.063799            0.0   
DCN                   4.72361  0.996138      0.418879           

In [17]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is your initial DataFrame
# Split data
X = df.drop('Y', axis=1)
y = df['Y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.FloatTensor(y_train.values).unsqueeze(1).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.FloatTensor(y_test.values).unsqueeze(1).to(device)

class WideAndDeepNetwork(nn.Module):
    def __init__(self, input_dim, wide_dim, deep_dims):
        super(WideAndDeepNetwork, self).__init__()
        self.wide = nn.Linear(input_dim, wide_dim)
        
        deep_layers = []
        prev_dim = input_dim
        for dim in deep_dims:
            deep_layers.append(nn.Linear(prev_dim, dim))
            deep_layers.append(nn.ReLU())
            prev_dim = dim
        self.deep = nn.Sequential(*deep_layers)
        
        self.final = nn.Linear(wide_dim + deep_dims[-1], 1)

    def forward(self, x):
        wide_out = self.wide(x)
        deep_out = self.deep(x)
        combined = torch.cat([wide_out, deep_out], dim=1)
        return self.final(combined)

def objective(trial):
    # Define hyperparameters to tune
    wide_dim = trial.suggest_int('wide_dim', 8, 128)
    n_deep_layers = trial.suggest_int('n_deep_layers', 1, 5)
    deep_dims = [trial.suggest_int(f'deep_dim_{i}', 32, 256) for i in range(n_deep_layers)]
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the Wide & Deep model
    model = WideAndDeepNetwork(X_train.shape[1], wide_dim, deep_dims).to(device)

    # Define loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test_tensor)
        mse = mean_squared_error(y_test_tensor.cpu().numpy(), y_pred.cpu().numpy())
        rmse = np.sqrt(mse)

    return rmse

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final Wide & Deep model with the best hyperparameters
best_model = WideAndDeepNetwork(X_train.shape[1], 
                                best_params['wide_dim'], 
                                [best_params[f'deep_dim_{i}'] for i in range(best_params['n_deep_layers'])]).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    y_pred = best_model(X_test_tensor)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_true = y_test_tensor.cpu().numpy()
    y_pred = y_pred.cpu().numpy()
    
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['Wide & Deep'] = [rmse, r2, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")


[I 2024-07-26 08:10:15,159] A new study created in memory with name: no-name-c86b482c-8e21-4561-bf41-f15418b31c04


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\3817342284.py:63: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 08:10:15,432] Trial 0 finished with value: 6.078895092010498 and parameters: {'wide_dim': 92, 'n_deep_layers': 3, 'deep_dim_0': 142, 'deep_dim_1': 37, 'deep_dim_2': 207, 'learning_rate': 0.0791549434665011, 'batch_size': 64, 'num_epochs': 53}. Best is trial 0 with value: 6.078895092010498.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\3817342284.py:63: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024

                         RMSE R-squared Training Time Inference Time  \
Linear Regression    0.211628  0.999992      0.001003            0.0   
Ridge Regression     0.905454  0.999858      0.000999            0.0   
Lasso Regression     0.383952  0.999974           0.0            0.0   
ElasticNet           4.404609  0.996642      0.001995            0.0   
Decision Tree       36.807797    0.7655      0.000998            0.0   
Random Forest       37.345571  0.758598      0.391953       0.023935   
Gradient Boosting   32.551013  0.816603      0.060839       0.000996   
XGBoost             47.556448  0.608545      0.066822       0.000997   
LightGBM           105.104923 -0.912095      0.005985       0.000997   
CatBoost             61.82527    0.3384       0.16257       0.001988   
MLP                  7.094997  0.991287      0.594412       0.000998   
DNN                109.746297 -1.084697      0.063799            0.0   
DCN                   4.72361  0.996138      0.418879           

In [18]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import optuna
import xgboost as xgb

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is your initial DataFrame
# Split data
X = df.drop('Y', axis=1)
y = df['Y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.FloatTensor(y_train.values).unsqueeze(1).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.FloatTensor(y_test.values).unsqueeze(1).to(device)

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dims):
        super(NeuralNetwork, self).__init__()
        layers = []
        prev_dim = input_dim
        for dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, dim))
            layers.append(nn.ReLU())
            prev_dim = dim
        layers.append(nn.Linear(prev_dim, 1))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

def objective(trial):
    # Define hyperparameters to tune for XGBoost
    xgb_params = {
        'n_estimators': trial.suggest_int('xgb_n_estimators', 50, 200),
        'max_depth': trial.suggest_int('xgb_max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('xgb_learning_rate', 1e-4, 1e-1),
        'subsample': trial.suggest_uniform('xgb_subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('xgb_colsample_bytree', 0.5, 1.0)
    }

    # Train XGBoost model
    xgb_model = xgb.XGBRegressor(**xgb_params)
    xgb_model.fit(X_train_scaled, y_train)

    # Transform the data using the XGBoost model
    X_train_transformed = xgb_model.apply(X_train_scaled)
    X_test_transformed = xgb_model.apply(X_test_scaled)

    # Define hyperparameters to tune for Neural Network
    n_layers = trial.suggest_int('nn_n_layers', 1, 5)
    hidden_dims = [trial.suggest_int(f'nn_hidden_dim_{i}', 32, 256) for i in range(n_layers)]
    nn_learning_rate = trial.suggest_loguniform('nn_learning_rate', 1e-4, 1e-1)
    nn_batch_size = trial.suggest_categorical('nn_batch_size', [32, 64, 128, 256])
    nn_num_epochs = trial.suggest_int('nn_num_epochs', 10, 100)

    # Create the Neural Network model
    nn_model = NeuralNetwork(X_train_transformed.shape[1], hidden_dims).to(device)

    # Define loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(nn_model.parameters(), lr=nn_learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(torch.FloatTensor(X_train_transformed).to(device), y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=nn_batch_size, shuffle=True)

    # Training
    for epoch in range(nn_num_epochs):
        nn_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = nn_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    nn_model.eval()
    with torch.no_grad():
        y_pred = nn_model(torch.FloatTensor(X_test_transformed).to(device))
        mse = mean_squared_error(y_test_tensor.cpu().numpy(), y_pred.cpu().numpy())
        rmse = np.sqrt(mse)

    return rmse

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final XGBoost model with the best hyperparameters
xgb_params = {
    'n_estimators': best_params['xgb_n_estimators'],
    'max_depth': best_params['xgb_max_depth'],
    'learning_rate': best_params['xgb_learning_rate'],
    'subsample': best_params['xgb_subsample'],
    'colsample_bytree': best_params['xgb_colsample_bytree']
}
xgb_model = xgb.XGBRegressor(**xgb_params)
xgb_model.fit(X_train_scaled, y_train)

# Transform the data using the XGBoost model
X_train_transformed = xgb_model.apply(X_train_scaled)
X_test_transformed = xgb_model.apply(X_test_scaled)

# Train the final Neural Network model with the best hyperparameters
nn_model = NeuralNetwork(X_train_transformed.shape[1], 
                         [best_params[f'nn_hidden_dim_{i}'] for i in range(best_params['nn_n_layers'])]).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(nn_model.parameters(), lr=best_params['nn_learning_rate'])

train_dataset = TensorDataset(torch.FloatTensor(X_train_transformed).to(device), y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['nn_batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(best_params['nn_num_epochs']):
    nn_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = nn_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
nn_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    y_pred = nn_model(torch.FloatTensor(X_test_transformed).to(device))
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_true = y_test_tensor.cpu().numpy()
    y_pred = y_pred.cpu().numpy()
    
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['XGBoost + NN'] = [rmse, r2, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")


[I 2024-07-26 08:11:32,383] A new study created in memory with name: no-name-02df36d3-dd1e-4742-a6eb-fd488a87cb4a
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\2800434033.py:58: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('xgb_learning_rate', 1e-4, 1e-1),
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\2800434033.py:59: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('xgb_subsample', 0.5, 1.0),
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\2800434033.py:60: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/ta

Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\2800434033.py:58: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('xgb_learning_rate', 1e-4, 1e-1),
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\2800434033.py:59: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('xgb_subsample', 0.5, 1.0),
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\2800434033.py:60: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('xgb_colsample_bytree', 0.5, 1.0)

                         RMSE R-squared Training Time Inference Time  \
Linear Regression    0.211628  0.999992      0.001003            0.0   
Ridge Regression     0.905454  0.999858      0.000999            0.0   
Lasso Regression     0.383952  0.999974           0.0            0.0   
ElasticNet           4.404609  0.996642      0.001995            0.0   
Decision Tree       36.807797    0.7655      0.000998            0.0   
Random Forest       37.345571  0.758598      0.391953       0.023935   
Gradient Boosting   32.551013  0.816603      0.060839       0.000996   
XGBoost             47.556448  0.608545      0.066822       0.000997   
LightGBM           105.104923 -0.912095      0.005985       0.000997   
CatBoost             61.82527    0.3384       0.16257       0.001988   
MLP                  7.094997  0.991287      0.594412       0.000998   
DNN                109.746297 -1.084697      0.063799            0.0   
DCN                   4.72361  0.996138      0.418879           

In [20]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import optuna
import lightgbm as lgb

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is your initial DataFrame
# Split data
X = df.drop('Y', axis=1)
y = df['Y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.FloatTensor(y_train.values).unsqueeze(1).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.FloatTensor(y_test.values).unsqueeze(1).to(device)

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dims):
        super(NeuralNetwork, self).__init__()
        layers = []
        prev_dim = input_dim
        for dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, dim))
            layers.append(nn.ReLU())
            prev_dim = dim
        layers.append(nn.Linear(prev_dim, 1))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

def objective(trial):
    # Define hyperparameters to tune for LightGBM
    lgb_params = {
        'n_estimators': trial.suggest_int('lgb_n_estimators', 50, 200),
        'max_depth': trial.suggest_int('lgb_max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('lgb_learning_rate', 1e-4, 1e-1),
        'subsample': trial.suggest_uniform('lgb_subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('lgb_colsample_bytree', 0.5, 1.0)
    }

    # Train LightGBM model
    lgb_model = lgb.LGBMRegressor(**lgb_params)
    lgb_model.fit(X_train_scaled, y_train)

    # Transform the data using the LightGBM model (use raw predictions as features)
    X_train_transformed = lgb_model.predict(X_train_scaled, raw_score=True).reshape(-1, 1)
    X_test_transformed = lgb_model.predict(X_test_scaled, raw_score=True).reshape(-1, 1)

    # Define hyperparameters to tune for Neural Network
    n_layers = trial.suggest_int('nn_n_layers', 1, 5)
    hidden_dims = [trial.suggest_int(f'nn_hidden_dim_{i}', 32, 256) for i in range(n_layers)]
    nn_learning_rate = trial.suggest_loguniform('nn_learning_rate', 1e-4, 1e-1)
    nn_batch_size = trial.suggest_categorical('nn_batch_size', [32, 64, 128, 256])
    nn_num_epochs = trial.suggest_int('nn_num_epochs', 10, 100)

    # Create the Neural Network model
    nn_model = NeuralNetwork(X_train_transformed.shape[1], hidden_dims).to(device)

    # Define loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(nn_model.parameters(), lr=nn_learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(torch.FloatTensor(X_train_transformed).to(device), y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=nn_batch_size, shuffle=True)

    # Training
    for epoch in range(nn_num_epochs):
        nn_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = nn_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    nn_model.eval()
    with torch.no_grad():
        y_pred = nn_model(torch.FloatTensor(X_test_transformed).to(device))
        mse = mean_squared_error(y_test_tensor.cpu().numpy(), y_pred.cpu().numpy())
        rmse = np.sqrt(mse)

    return rmse

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final LightGBM model with the best hyperparameters
lgb_params = {
    'n_estimators': best_params['lgb_n_estimators'],
    'max_depth': best_params['lgb_max_depth'],
    'learning_rate': best_params['lgb_learning_rate'],
    'subsample': best_params['lgb_subsample'],
    'colsample_bytree': best_params['lgb_colsample_bytree']
}
lgb_model = lgb.LGBMRegressor(**lgb_params)
lgb_model.fit(X_train_scaled, y_train)

# Transform the data using the LightGBM model (use raw predictions as features)
X_train_transformed = lgb_model.predict(X_train_scaled, raw_score=True).reshape(-1, 1)
X_test_transformed = lgb_model.predict(X_test_scaled, raw_score=True).reshape(-1, 1)

# Train the final Neural Network model with the best hyperparameters
nn_model = NeuralNetwork(X_train_transformed.shape[1], 
                         [best_params[f'nn_hidden_dim_{i}'] for i in range(best_params['nn_n_layers'])]).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(nn_model.parameters(), lr=best_params['nn_learning_rate'])

train_dataset = TensorDataset(torch.FloatTensor(X_train_transformed).to(device), y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['nn_batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(best_params['nn_num_epochs']):
    nn_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = nn_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
nn_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    y_pred = nn_model(torch.FloatTensor(X_test_transformed).to(device))
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_true = y_test_tensor.cpu().numpy()
    y_pred = y_pred.cpu().numpy()
    
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['LightGBM + NN'] = [rmse, r2, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")


[I 2024-07-26 08:16:22,655] A new study created in memory with name: no-name-c8fd92fb-64cd-4842-99f7-ebcb28058e2d


Using device: cpu
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 24, number of used features: 0
[LightGBM] [Info] Start training from score 7.791087
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves 

C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\1234451331.py:58: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('lgb_learning_rate', 1e-4, 1e-1),
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\1234451331.py:59: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('lgb_subsample', 0.5, 1.0),
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\1234451331.py:60: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('lgb_colsample_bytree', 0.5, 1.0)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 24, number of used features: 0
[LightGBM] [Info] Start training from score 7.791087
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the spli

[I 2024-07-26 08:16:23,304] Trial 2 finished with value: 106.50305938720703 and parameters: {'lgb_n_estimators': 158, 'lgb_max_depth': 4, 'lgb_learning_rate': 0.038195058985550756, 'lgb_subsample': 0.646086635863193, 'lgb_colsample_bytree': 0.9555746204783562, 'nn_n_layers': 5, 'nn_hidden_dim_0': 218, 'nn_hidden_dim_1': 51, 'nn_hidden_dim_2': 65, 'nn_hidden_dim_3': 98, 'nn_hidden_dim_4': 209, 'nn_learning_rate': 0.017250623908633172, 'nn_batch_size': 128, 'nn_num_epochs': 27}. Best is trial 0 with value: 104.6399154663086.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\1234451331.py:58: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('lgb_learning_rate', 1e-4, 1e-1),
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\1234451331.py:59: FutureWarning: suggest_uniform has been deprecated in 

...

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 24, number of used features: 0
[LightGBM] [Info] Start training from score 7.791087
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the spli

C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\1234451331.py:58: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('lgb_learning_rate', 1e-4, 1e-1),
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\1234451331.py:59: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('lgb_subsample', 0.5, 1.0),
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\1234451331.py:60: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('lgb_colsample_bytree', 0.5, 1.0)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 24, number of used features: 0
[LightGBM] [Info] Start training from score 7.791087
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the spli

C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\1234451331.py:58: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('lgb_learning_rate', 1e-4, 1e-1),
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\1234451331.py:59: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('lgb_subsample', 0.5, 1.0),
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\1234451331.py:60: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('lgb_colsample_bytree', 0.5, 1.0)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 24, number of used features: 0
[LightGBM] [Info] Start training from score 7.791087
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the spli

In [21]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import optuna
from catboost import CatBoostRegressor, Pool

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is your initial DataFrame
# Split data
X = df.drop('Y', axis=1)
y = df['Y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.FloatTensor(y_train.values).unsqueeze(1).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.FloatTensor(y_test.values).unsqueeze(1).to(device)

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dims):
        super(NeuralNetwork, self).__init__()
        layers = []
        prev_dim = input_dim
        for dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, dim))
            layers.append(nn.ReLU())
            prev_dim = dim
        layers.append(nn.Linear(prev_dim, 1))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

def objective(trial):
    # Define hyperparameters to tune for CatBoost
    cat_params = {
        'iterations': trial.suggest_int('cat_iterations', 100, 1000),
        'depth': trial.suggest_int('cat_depth', 4, 10),
        'learning_rate': trial.suggest_loguniform('cat_learning_rate', 1e-3, 1),
        'l2_leaf_reg': trial.suggest_loguniform('cat_l2_leaf_reg', 1e-8, 10),
        'border_count': trial.suggest_int('cat_border_count', 32, 255),
        'bagging_temperature': trial.suggest_loguniform('cat_bagging_temperature', 0.01, 100.0)
    }

    # Train CatBoost model
    cat_model = CatBoostRegressor(**cat_params, verbose=False)
    cat_model.fit(X_train_scaled, y_train)

    # Transform the data using the CatBoost model
    X_train_transformed = cat_model.calc_leaf_indexes(X_train_scaled)
    X_test_transformed = cat_model.calc_leaf_indexes(X_test_scaled)

    # Define hyperparameters to tune for Neural Network
    n_layers = trial.suggest_int('nn_n_layers', 1, 5)
    hidden_dims = [trial.suggest_int(f'nn_hidden_dim_{i}', 32, 256) for i in range(n_layers)]
    nn_learning_rate = trial.suggest_loguniform('nn_learning_rate', 1e-4, 1e-1)
    nn_batch_size = trial.suggest_categorical('nn_batch_size', [32, 64, 128, 256])
    nn_num_epochs = trial.suggest_int('nn_num_epochs', 10, 100)

    # Create the Neural Network model
    nn_model = NeuralNetwork(X_train_transformed.shape[1], hidden_dims).to(device)

    # Define loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(nn_model.parameters(), lr=nn_learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(torch.FloatTensor(X_train_transformed).to(device), y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=nn_batch_size, shuffle=True)

    # Training
    for epoch in range(nn_num_epochs):
        nn_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = nn_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    nn_model.eval()
    with torch.no_grad():
        y_pred = nn_model(torch.FloatTensor(X_test_transformed).to(device))
        mse = mean_squared_error(y_test_tensor.cpu().numpy(), y_pred.cpu().numpy())
        rmse = np.sqrt(mse)

    return rmse

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final CatBoost model with the best hyperparameters
cat_params = {
    'iterations': best_params['cat_iterations'],
    'depth': best_params['cat_depth'],
    'learning_rate': best_params['cat_learning_rate'],
    'l2_leaf_reg': best_params['cat_l2_leaf_reg'],
    'border_count': best_params['cat_border_count'],
    'bagging_temperature': best_params['cat_bagging_temperature']
}
cat_model = CatBoostRegressor(**cat_params, verbose=False)
cat_model.fit(X_train_scaled, y_train)

# Transform the data using the CatBoost model
X_train_transformed = cat_model.calc_leaf_indexes(X_train_scaled)
X_test_transformed = cat_model.calc_leaf_indexes(X_test_scaled)

# Train the final Neural Network model with the best hyperparameters
nn_model = NeuralNetwork(X_train_transformed.shape[1], 
                         [best_params[f'nn_hidden_dim_{i}'] for i in range(best_params['nn_n_layers'])]).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(nn_model.parameters(), lr=best_params['nn_learning_rate'])

train_dataset = TensorDataset(torch.FloatTensor(X_train_transformed).to(device), y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['nn_batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(best_params['nn_num_epochs']):
    nn_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = nn_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
nn_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    y_pred = nn_model(torch.FloatTensor(X_test_transformed).to(device))
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_true = y_test_tensor.cpu().numpy()
    y_pred = y_pred.cpu().numpy()
    
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['CatBoost + NN'] = [rmse, r2, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")


[I 2024-07-26 08:16:57,374] A new study created in memory with name: no-name-add0a9c7-a0a3-48c5-81ab-2b05f7fdf1b2


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\2959969898.py:58: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('cat_learning_rate', 1e-3, 1),
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\2959969898.py:59: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('cat_l2_leaf_reg', 1e-8, 10),
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\2959969898.py:61: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'bagging_temperature': trial.sugges

                         RMSE R-squared Training Time Inference Time  \
Linear Regression    0.211628  0.999992      0.001003            0.0   
Ridge Regression     0.905454  0.999858      0.000999            0.0   
Lasso Regression     0.383952  0.999974           0.0            0.0   
ElasticNet           4.404609  0.996642      0.001995            0.0   
Decision Tree       36.807797    0.7655      0.000998            0.0   
Random Forest       37.345571  0.758598      0.391953       0.023935   
Gradient Boosting   32.551013  0.816603      0.060839       0.000996   
XGBoost             47.556448  0.608545      0.066822       0.000997   
LightGBM           105.104923 -0.912095      0.005985       0.000997   
CatBoost             61.82527    0.3384       0.16257       0.001988   
MLP                  7.094997  0.991287      0.594412       0.000998   
DNN                109.746297 -1.084697      0.063799            0.0   
DCN                   4.72361  0.996138      0.418879           

In [26]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is your initial DataFrame
# Split data
X = df.drop('Y', axis=1)
y = df['Y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.FloatTensor(y_train.values).unsqueeze(1).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.FloatTensor(y_test.values).unsqueeze(1).to(device)

class AutoInt(nn.Module):
    def __init__(self, input_dim, embedding_dim, num_heads, num_layers, dropout):
        super(AutoInt, self).__init__()
        self.embedding = nn.Linear(input_dim, embedding_dim)
        self.multi_head_attentions = nn.ModuleList([
            nn.MultiheadAttention(embedding_dim, num_heads, dropout=dropout)
            for _ in range(num_layers)
        ])
        self.fc = nn.Linear(embedding_dim, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.embedding(x).unsqueeze(0)  # Add sequence dimension
        for attention in self.multi_head_attentions:
            x, _ = attention(x, x, x)
        x = x.squeeze(0)  # Remove sequence dimension
        x = self.dropout(x)
        return self.fc(x)

def objective(trial):
    # Define hyperparameters to tune
    num_heads = trial.suggest_int('num_heads', 1, 8)
    embedding_dim = trial.suggest_int('embedding_dim', num_heads, 256, step=num_heads)  # Ensure divisibility
    num_layers = trial.suggest_int('num_layers', 1, 5)
    dropout = trial.suggest_float('dropout', 0.0, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the AutoInt model
    model = AutoInt(X_train.shape[1], embedding_dim, num_heads, num_layers, dropout).to(device)

    # Define loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test_tensor)
        mse = mean_squared_error(y_test_tensor.cpu().numpy(), y_pred.cpu().numpy())
        rmse = np.sqrt(mse)

    return rmse

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final AutoInt model with the best hyperparameters
best_model = AutoInt(X_train.shape[1], 
                     best_params['embedding_dim'], 
                     best_params['num_heads'], 
                     best_params['num_layers'], 
                     best_params['dropout']).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    y_pred = best_model(X_test_tensor)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_true = y_test_tensor.cpu().numpy()
    y_pred = y_pred.cpu().numpy()
    
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['AutoInt'] = [rmse, r2, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")


[I 2024-07-26 08:23:37,592] A new study created in memory with name: no-name-72a72849-e5aa-409c-9409-fb80cc019d2b


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\2454648724.py:62: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 08:23:39,866] Trial 0 finished with value: 53.65874481201172 and parameters: {'num_heads': 8, 'embedding_dim': 216, 'num_layers': 5, 'dropout': 0.33285250940195116, 'learning_rate': 0.00016484901891047237, 'batch_size': 128, 'num_epochs': 83}. Best is trial 0 with value: 53.65874481201172.
c:\Users\Tommy\anaconda3\Lib\site-packages\optuna\distributions.py:700: UserWarning: The distribution is specified by [7, 256] and step=7, but the range is not divisible by `step`. It will be replaced by [7, 252].
  warnings.warn(
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\2454648724.py:62: FutureWarning: suggest_loguniform has been deprecated

                         RMSE R-squared Training Time Inference Time  \
Linear Regression    0.211628  0.999992      0.001003            0.0   
Ridge Regression     0.905454  0.999858      0.000999            0.0   
Lasso Regression     0.383952  0.999974           0.0            0.0   
ElasticNet           4.404609  0.996642      0.001995            0.0   
Decision Tree       36.807797    0.7655      0.000998            0.0   
Random Forest       37.345571  0.758598      0.391953       0.023935   
Gradient Boosting   32.551013  0.816603      0.060839       0.000996   
XGBoost             47.556448  0.608545      0.066822       0.000997   
LightGBM           105.104923 -0.912095      0.005985       0.000997   
CatBoost             61.82527    0.3384       0.16257       0.001988   
MLP                  7.094997  0.991287      0.594412       0.000998   
DNN                109.746297 -1.084697      0.063799            0.0   
DCN                   4.72361  0.996138      0.418879           

In [25]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is your initial DataFrame
# Split data
X = df.drop('Y', axis=1)
y = df['Y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.FloatTensor(y_train.values).unsqueeze(1).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.FloatTensor(y_test.values).unsqueeze(1).to(device)

class FTTransformer(nn.Module):
    def __init__(self, input_dim, num_tokens, dim, depth, heads, mlp_dim, dropout=0.1):
        super(FTTransformer, self).__init__()
        self.num_tokens = num_tokens
        self.input_dim = input_dim
        self.token_embedding = nn.Embedding(num_tokens * input_dim, dim)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=dim, nhead=heads, dim_feedforward=mlp_dim, dropout=dropout),
            num_layers=depth
        )
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, 1)
        )

    def forward(self, x):
        # Tokenize the input
        x = (x * (self.num_tokens - 1)).long()
        x = torch.clamp(x, 0, self.num_tokens - 1)  # Ensure indices are within range
        x = x + torch.arange(x.shape[1], device=x.device) * self.num_tokens
        x = self.token_embedding(x)
        
        # Apply transformer
        x = self.transformer(x.permute(1, 0, 2)).permute(1, 0, 2)
        
        # Pool and predict
        x = x.mean(dim=1)
        return self.mlp_head(x)

def objective(trial):
    # Define hyperparameters to tune
    num_tokens = trial.suggest_int('num_tokens', 8, 256)
    heads = trial.suggest_int('heads', 1, 8)
    dim = trial.suggest_int('dim', heads, 256, step=heads)  # Ensure dim is divisible by heads
    depth = trial.suggest_int('depth', 1, 6)
    mlp_dim = trial.suggest_int('mlp_dim', 32, 256)
    dropout = trial.suggest_float('dropout', 0.0, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the FT Transformer model
    model = FTTransformer(X_train.shape[1], num_tokens, dim, depth, heads, mlp_dim, dropout).to(device)

    # Define loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test_tensor)
        mse = mean_squared_error(y_test_tensor.cpu().numpy(), y_pred.cpu().numpy())
        rmse = np.sqrt(mse)

    return rmse

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final FT Transformer model with the best hyperparameters
best_model = FTTransformer(X_train.shape[1], 
                           best_params['num_tokens'], 
                           best_params['dim'], 
                           best_params['depth'], 
                           best_params['heads'], 
                           best_params['mlp_dim'], 
                           best_params['dropout']).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    y_pred = best_model(X_test_tensor)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_true = y_test_tensor.cpu().numpy()
    y_pred = y_pred.cpu().numpy()
    
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['FT-Transformer'] = [rmse, r2, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")


[I 2024-07-26 08:21:41,082] A new study created in memory with name: no-name-c47986e0-ae22-4f5f-baba-0d189ed3e6a9


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\1004955102.py:74: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
c:\Users\Tommy\anaconda3\Lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
[I 2024-07-26 08:21:41,622] Trial 0 finished with value: 103.96244812011719 and parameters: {'num_tokens': 58, 'heads': 4, 'dim': 52, 'depth': 1, 'mlp_dim': 90, 'dropout': 0.4294006564307398, 'learning_rate': 0.0018117934028063763, 'batch_size': 64, 'num_epochs': 45}

                         RMSE R-squared Training Time Inference Time  \
Linear Regression    0.211628  0.999992      0.001003            0.0   
Ridge Regression     0.905454  0.999858      0.000999            0.0   
Lasso Regression     0.383952  0.999974           0.0            0.0   
ElasticNet           4.404609  0.996642      0.001995            0.0   
Decision Tree       36.807797    0.7655      0.000998            0.0   
Random Forest       37.345571  0.758598      0.391953       0.023935   
Gradient Boosting   32.551013  0.816603      0.060839       0.000996   
XGBoost             47.556448  0.608545      0.066822       0.000997   
LightGBM           105.104923 -0.912095      0.005985       0.000997   
CatBoost             61.82527    0.3384       0.16257       0.001988   
MLP                  7.094997  0.991287      0.594412       0.000998   
DNN                109.746297 -1.084697      0.063799            0.0   
DCN                   4.72361  0.996138      0.418879           

In [27]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is your initial DataFrame
# Split data
X = df.drop('Y', axis=1)
y = df['Y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.FloatTensor(y_train.values).unsqueeze(1).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.FloatTensor(y_test.values).unsqueeze(1).to(device)

class DynamicNet(nn.Module):
    def __init__(self, input_dim, layer_sizes, activations):
        super(DynamicNet, self).__init__()
        layers = []
        prev_dim = input_dim
        for size, activation in zip(layer_sizes, activations):
            layers.append(nn.Linear(prev_dim, size))
            if activation == 'relu':
                layers.append(nn.ReLU())
            elif activation == 'tanh':
                layers.append(nn.Tanh())
            elif activation == 'sigmoid':
                layers.append(nn.Sigmoid())
            prev_dim = size
        layers.append(nn.Linear(prev_dim, 1))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

def objective(trial):
    # Define hyperparameters to tune
    n_layers = trial.suggest_int('n_layers', 1, 5)
    layer_sizes = [trial.suggest_int(f'n_units_l{i}', 32, 256) for i in range(n_layers)]
    activations = [trial.suggest_categorical(f'activation_l{i}', ['relu', 'tanh', 'sigmoid']) for i in range(n_layers)]
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the model
    model = DynamicNet(X_train.shape[1], layer_sizes, activations).to(device)

    # Define loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test_tensor)
        mse = mean_squared_error(y_test_tensor.cpu().numpy(), y_pred.cpu().numpy())
        rmse = np.sqrt(mse)

    return rmse

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final model with the best hyperparameters
layer_sizes = [best_params[f'n_units_l{i}'] for i in range(best_params['n_layers'])]
activations = [best_params[f'activation_l{i}'] for i in range(best_params['n_layers'])]
best_model = DynamicNet(X_train.shape[1], layer_sizes, activations).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    y_pred = best_model(X_test_tensor)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_true = y_test_tensor.cpu().numpy()
    y_pred = y_pred.cpu().numpy()
    
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['NAS'] = [rmse, r2, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")


[I 2024-07-26 08:26:00,400] A new study created in memory with name: no-name-3e541fa6-fe18-4b91-b923-1b72e5976d50


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\1217925743.py:62: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 08:26:00,616] Trial 0 finished with value: 87.46815490722656 and parameters: {'n_layers': 5, 'n_units_l0': 201, 'n_units_l1': 170, 'n_units_l2': 158, 'n_units_l3': 145, 'n_units_l4': 221, 'activation_l0': 'tanh', 'activation_l1': 'sigmoid', 'activation_l2': 'relu', 'activation_l3': 'sigmoid', 'activation_l4': 'sigmoid', 'learning_rate': 0.010669005980628142, 'batch_size': 64, 'num_epochs': 18}. Best is trial 0 with value: 87.46815490722656.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\1217925743.py:62: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/op

                         RMSE R-squared Training Time Inference Time  \
Linear Regression    0.211628  0.999992      0.001003            0.0   
Ridge Regression     0.905454  0.999858      0.000999            0.0   
Lasso Regression     0.383952  0.999974           0.0            0.0   
ElasticNet           4.404609  0.996642      0.001995            0.0   
Decision Tree       36.807797    0.7655      0.000998            0.0   
Random Forest       37.345571  0.758598      0.391953       0.023935   
Gradient Boosting   32.551013  0.816603      0.060839       0.000996   
XGBoost             47.556448  0.608545      0.066822       0.000997   
LightGBM           105.104923 -0.912095      0.005985       0.000997   
CatBoost             61.82527    0.3384       0.16257       0.001988   
MLP                  7.094997  0.991287      0.594412       0.000998   
DNN                109.746297 -1.084697      0.063799            0.0   
DCN                   4.72361  0.996138      0.418879           

In [38]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torchdiffeq import odeint_adjoint as odeint
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is your initial DataFrame
# Split data
X = df.drop('Y', axis=1)
y = df['Y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.FloatTensor(y_train.values).unsqueeze(1).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.FloatTensor(y_test.values).unsqueeze(1).to(device)

class ODEFunc(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, activation):
        super(ODEFunc, self).__init__()
        layers = []
        prev_dim = input_dim
        for _ in range(num_layers):
            layers.append(nn.Linear(prev_dim, hidden_dim))
            if activation == 'relu':
                layers.append(nn.ReLU())
            elif activation == 'tanh':
                layers.append(nn.Tanh())
            elif activation == 'sigmoid':
                layers.append(nn.Sigmoid())
            prev_dim = hidden_dim
        layers.append(nn.Linear(prev_dim, input_dim))
        self.network = nn.Sequential(*layers)

    def forward(self, t, x):
        return self.network(x)

class NODE(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, activation):
        super(NODE, self).__init__()
        self.odefunc = ODEFunc(input_dim, hidden_dim, num_layers, activation)
        self.fc = nn.Linear(input_dim, 1)

    def forward(self, x):
        t = torch.tensor([0, 1], dtype=torch.float32).to(x.device)
        out = odeint(self.odefunc, x, t, method='rk4')[-1]
        return self.fc(out)

def objective(trial):
    # Define hyperparameters to tune
    hidden_dim = trial.suggest_int('hidden_dim', 32, 128)
    num_layers = trial.suggest_int('num_layers', 1, 3)
    activation = trial.suggest_categorical('activation', ['relu', 'tanh', 'sigmoid'])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    num_epochs = trial.suggest_int('num_epochs', 10, 30)

    # Create the NODE model
    model = NODE(X_train.shape[1], hidden_dim, num_layers, activation).to(device)

    # Define loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test_tensor)
        mse = mean_squared_error(y_test_tensor.cpu().numpy(), y_pred.cpu().numpy())
        rmse = np.sqrt(mse)

    return rmse

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)  # Reduced number of trials for faster tuning

# Get the best hyperparameters
best_params = study.best_params

# Train the final NODE model with the best hyperparameters
best_model = NODE(X_train.shape[1], 
                  best_params['hidden_dim'], 
                  best_params['num_layers'], 
                  best_params['activation']).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    y_pred = best_model(X_test_tensor)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_true = y_test_tensor.cpu().numpy()
    y_pred = y_pred.cpu().numpy()
    
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['NODE'] = [rmse, r2, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")


[I 2024-07-26 08:47:05,834] A new study created in memory with name: no-name-357476c1-4ae8-493f-94fd-34b393b5fb8b


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\4231878458.py:74: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
[I 2024-07-26 08:47:06,483] Trial 0 finished with value: 110.83451080322266 and parameters: {'hidden_dim': 96, 'num_layers': 3, 'activation': 'sigmoid', 'learning_rate': 0.00019913092117633768, 'batch_size': 32, 'num_epochs': 25}. Best is trial 0 with value: 110.83451080322266.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\4231878458.py:74: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
[I 2024-07-26 08:47:06,730] Trial

                         RMSE R-squared Training Time Inference Time  \
Linear Regression    0.211628  0.999992      0.001003            0.0   
Ridge Regression     0.905454  0.999858      0.000999            0.0   
Lasso Regression     0.383952  0.999974           0.0            0.0   
ElasticNet           4.404609  0.996642      0.001995            0.0   
Decision Tree       36.807797    0.7655      0.000998            0.0   
Random Forest       37.345571  0.758598      0.391953       0.023935   
Gradient Boosting   32.551013  0.816603      0.060839       0.000996   
XGBoost             47.556448  0.608545      0.066822       0.000997   
LightGBM           105.104923 -0.912095      0.005985       0.000997   
CatBoost             61.82527    0.3384       0.16257       0.001988   
MLP                  7.094997  0.991287      0.594412       0.000998   
DNN                109.746297 -1.084697      0.063799            0.0   
DCN                   4.72361  0.996138      0.418879           

In [35]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import optuna
from pytorch_tabnet.tab_model import TabNetRegressor

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is your initial DataFrame
# Split data
X = df.drop('Y', axis=1)
y = df['Y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape y_train and y_test to 2D arrays
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

def objective(trial):
    # Define hyperparameters to tune
    n_d = trial.suggest_int('n_d', 8, 64)
    n_a = trial.suggest_int('n_a', 8, 64)
    n_steps = trial.suggest_int('n_steps', 3, 10)
    gamma = trial.suggest_float('gamma', 1.0, 2.0)
    lambda_sparse = trial.suggest_float('lambda_sparse', 1e-6, 1e-3)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the TabNet model
    model = TabNetRegressor(
        n_d=n_d, n_a=n_a, n_steps=n_steps, gamma=gamma, lambda_sparse=lambda_sparse,
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=learning_rate),
        scheduler_params={"step_size":50, "gamma":0.9},
        scheduler_fn=torch.optim.lr_scheduler.StepLR,
        mask_type='entmax'
    )

    # Training
    model.fit(
        X_train_scaled, y_train,
        eval_set=[(X_test_scaled, y_test)],
        max_epochs=num_epochs,
        patience=5,
        batch_size=batch_size,
        virtual_batch_size=batch_size // 4,
        num_workers=0,
        drop_last=False
    )

    # Evaluation
    y_pred = model.predict(X_test_scaled)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    return rmse

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final TabNet model with the best hyperparameters
best_model = TabNetRegressor(
    n_d=best_params['n_d'],
    n_a=best_params['n_a'],
    n_steps=best_params['n_steps'],
    gamma=best_params['gamma'],
    lambda_sparse=best_params['lambda_sparse'],
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=best_params['learning_rate']),
    scheduler_params={"step_size":50, "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='entmax'
)

training_start_time = time.time()
best_model.fit(
    X_train_scaled, y_train,
    eval_set=[(X_test_scaled, y_test)],
    max_epochs=best_params['num_epochs'],
    patience=5,
    batch_size=best_params['batch_size'],
    virtual_batch_size=best_params['batch_size'] // 4,
    num_workers=0,
    drop_last=False
)
training_time = time.time() - training_start_time

# Evaluation
inference_start_time = time.time()
y_pred = best_model.predict(X_test_scaled)
inference_time = time.time() - inference_start_time

# Compute metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['TabNet'] = [rmse, r2, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")


[I 2024-07-26 08:39:52,078] A new study created in memory with name: no-name-c7a724b3-ec54-4626-8671-f94974c00f33


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\2903159880.py:43: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 8256.16309| val_0_mse: 23110.13626|  0:00:00s
epoch 1  | loss: 8055.23438| val_0_mse: 20637.32674|  0:00:00s
epoch 2  | loss: 7819.08838| val_0_mse: 17622.58602|  0:00:00s
epoch 3  | loss: 7459.15283| val_0_mse: 14274.93994|  0:00:00s
epoch 4  | loss: 7402.29443| val_0_mse: 13705.05766|  0:00:00s
epoch 5  | loss: 7144.45703| val_0_mse: 13284.58937|  0:00:00s
epoch 6  | loss: 6884.3125| val_0_mse: 12814.47813|  0:00:00s
epoch 7  | loss: 6592.36719| val_0_mse: 12400.12692|  0:00:01s
epoch 8  | loss: 6331.91016| val_0_mse: 12183.93229|  0:00:01s
epoch 9  | loss: 6030.80127| val_0_mse: 11871.92538|  0:00:01s
epoch 10 | loss: 5773.92188| val_0_mse: 11227.09959|  0:00:01s
epoch 11 | loss: 5508.97852| val_0_mse: 11209.23243|  0:00:01s
epoch 12 | loss: 5292.21338| val_0_mse: 11170.52595|  0:00:01s
epoch 13 | loss: 5037.39404| val_0_mse: 10963.63377|  0:00:01s
epoch 14 | loss: 4804.24707| val_0_mse: 10890.63483|  0:00:01s
epoch 15 | loss: 4662.29053| val_0_mse: 10832.59968|  0:

c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-26 08:39:54,465] Trial 0 finished with value: 103.30708498687908 and parameters: {'n_d': 63, 'n_a': 47, 'n_steps': 6, 'gamma': 1.3616047743555921, 'lambda_sparse': 0.0002613536321293988, 'learning_rate': 0.0029466101498928417, 'batch_size': 256, 'num_epochs': 17}. Best is trial 0 with value: 103.30708498687908.


...

c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-26 08:41:56,097] Trial 49 finished with value: 109.54779040742179 and parameters: {'n_d': 57, 'n_a': 17, 'n_steps': 5, 'gamma': 1.4793420463559992, 'lambda_sparse': 0.0008450546734865454, 'learning_rate': 0.0002658764086456467, 'batch_size': 256, 'num_epochs': 64}. Best is trial 23 with value: 20.873160944553998.


epoch 44 | loss: 7735.33984| val_0_mse: 12085.09503|  0:00:04s

Early stopping occurred at epoch 44 with best_epoch = 39 and best_val_0_mse = 12000.71838
epoch 0  | loss: 8633.33496| val_0_mse: 6839.38934|  0:00:00s


c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 1  | loss: 8273.11816| val_0_mse: 7378.03251|  0:00:00s
epoch 2  | loss: 7855.49609| val_0_mse: 7280.71715|  0:00:00s
epoch 3  | loss: 7432.57324| val_0_mse: 8492.0348|  0:00:00s
epoch 4  | loss: 7011.39258| val_0_mse: 8269.32929|  0:00:00s
epoch 5  | loss: 6462.71191| val_0_mse: 2769.09869|  0:00:00s
epoch 6  | loss: 6330.91406| val_0_mse: 2407.87857|  0:00:00s
epoch 7  | loss: 5603.52783| val_0_mse: 1757.27259|  0:00:00s
epoch 8  | loss: 5215.52246| val_0_mse: 1056.32327|  0:00:00s
epoch 9  | loss: 5177.76025| val_0_mse: 816.5443|  0:00:00s
epoch 10 | loss: 4581.8584| val_0_mse: 1285.6773|  0:00:00s
epoch 11 | loss: 4421.73877| val_0_mse: 1041.33952|  0:00:01s
epoch 12 | loss: 4181.3125| val_0_mse: 1351.37448|  0:00:01s
epoch 13 | loss: 3395.61475| val_0_mse: 1016.18194|  0:00:01s
epoch 14 | loss: 3319.2395| val_0_mse: 435.68885|  0:00:01s
epoch 15 | loss: 2820.70264| val_0_mse: 617.68171|  0:00:01s
epoch 16 | loss: 2626.62671| val_0_mse: 729.00361|  0:00:01s
epoch 17 | loss: 2

c:\Users\Tommy\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [30]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is your initial DataFrame
# Split data
X = df.drop('Y', axis=1)
y = df['Y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.FloatTensor(y_train.values).unsqueeze(1).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.FloatTensor(y_test.values).unsqueeze(1).to(device)

class KAN(nn.Module):
    def __init__(self, input_dim, inner_dim, outer_dim):
        super(KAN, self).__init__()
        self.inner_functions = nn.ModuleList([nn.Sequential(
            nn.Linear(1, inner_dim),
            nn.ReLU(),
            nn.Linear(inner_dim, 1)
        ) for _ in range(input_dim)])
        
        self.outer_function = nn.Sequential(
            nn.Linear(input_dim, outer_dim),
            nn.ReLU(),
            nn.Linear(outer_dim, 1)
        )

    def forward(self, x):
        inner_outputs = [f(x[:, i].unsqueeze(1)) for i, f in enumerate(self.inner_functions)]
        inner_concat = torch.cat(inner_outputs, dim=1)
        return self.outer_function(inner_concat)

def objective(trial):
    # Define hyperparameters to tune
    inner_dim = trial.suggest_int('inner_dim', 8, 64)
    outer_dim = trial.suggest_int('outer_dim', 8, 64)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the KAN model
    model = KAN(X_train.shape[1], inner_dim, outer_dim).to(device)

    # Define loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test_tensor)
        mse = mean_squared_error(y_test_tensor.cpu().numpy(), y_pred.cpu().numpy())
        rmse = np.sqrt(mse)

    return rmse

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final KAN model with the best hyperparameters
best_model = KAN(X_train.shape[1], best_params['inner_dim'], best_params['outer_dim']).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    y_pred = best_model(X_test_tensor)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_true = y_test_tensor.cpu().numpy()
    y_pred = y_pred.cpu().numpy()
    
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['KAN'] = [rmse, r2, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")


[I 2024-07-26 08:35:27,756] A new study created in memory with name: no-name-96957bfb-727e-4ca1-9508-7751ece5b423


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\1506177012.py:61: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 08:35:28,116] Trial 0 finished with value: 47.61610412597656 and parameters: {'inner_dim': 16, 'outer_dim': 47, 'learning_rate': 0.024741840873254223, 'batch_size': 128, 'num_epochs': 35}. Best is trial 0 with value: 47.61610412597656.
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\1506177012.py:61: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-07-26 08:35:28,573] Trial 1 finished with value: 33.10

                         RMSE R-squared Training Time Inference Time  \
Linear Regression    0.211628  0.999992      0.001003            0.0   
Ridge Regression     0.905454  0.999858      0.000999            0.0   
Lasso Regression     0.383952  0.999974           0.0            0.0   
ElasticNet           4.404609  0.996642      0.001995            0.0   
Decision Tree       36.807797    0.7655      0.000998            0.0   
Random Forest       37.345571  0.758598      0.391953       0.023935   
Gradient Boosting   32.551013  0.816603      0.060839       0.000996   
XGBoost             47.556448  0.608545      0.066822       0.000997   
LightGBM           105.104923 -0.912095      0.005985       0.000997   
CatBoost             61.82527    0.3384       0.16257       0.001988   
MLP                  7.094997  0.991287      0.594412       0.000998   
DNN                109.746297 -1.084697      0.063799            0.0   
DCN                   4.72361  0.996138      0.418879           

In [4]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is your initial DataFrame
# Split data
X = df.drop('Y', axis=1)
y = df['Y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.FloatTensor(y_train.values).unsqueeze(1).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.FloatTensor(y_test.values).unsqueeze(1).to(device)

class SAINT(nn.Module):
    def __init__(self, input_dim, embed_dim, num_heads, num_layers, dropout):
        super(SAINT, self).__init__()
        self.embedding = nn.Linear(input_dim, embed_dim)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dropout=dropout),
            num_layers=num_layers
        )
        self.fc = nn.Linear(embed_dim, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.embedding(x)
        x = x.unsqueeze(0)  # Add sequence dimension
        x = self.transformer(x)
        x = x.squeeze(0)  # Remove sequence dimension
        x = self.dropout(x)
        return self.fc(x)

def objective(trial):
    # Define hyperparameters to tune
    num_heads = trial.suggest_int('num_heads', 1, 8)
    embed_dim = trial.suggest_int('embed_dim', num_heads, 256, step=num_heads)  # Ensure divisibility
    num_layers = trial.suggest_int('num_layers', 1, 6)
    dropout = trial.suggest_float('dropout', 0.0, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)

    # Create the SAINT model
    model = SAINT(X_train.shape[1], embed_dim, num_heads, num_layers, dropout).to(device)

    # Define loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test_tensor)
        mse = mean_squared_error(y_test_tensor.cpu().numpy(), y_pred.cpu().numpy())
        rmse = np.sqrt(mse)

    return rmse

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final SAINT model with the best hyperparameters
best_model = SAINT(X_train.shape[1], 
                   best_params['embed_dim'], 
                   best_params['num_heads'], 
                   best_params['num_layers'], 
                   best_params['dropout']).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    y_pred = best_model(X_test_tensor)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_true = y_test_tensor.cpu().numpy()
    y_pred = y_pred.cpu().numpy()
    
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['SAINT'] = [rmse, r2, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")


[I 2024-07-26 08:50:32,296] A new study created in memory with name: no-name-6270cf44-c4c4-4bea-a86c-6125d3d0accf


Using device: cpu


c:\Users\Tommy\anaconda3\Lib\site-packages\optuna\distributions.py:700: UserWarning: The distribution is specified by [6, 256] and step=6, but the range is not divisible by `step`. It will be replaced by [6, 252].
  warnings.warn(
C:\Users\Tommy\AppData\Local\Temp\ipykernel_260\3223521253.py:62: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
c:\Users\Tommy\anaconda3\Lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
[I 2024-07-26 08:50:37,669] Trial 0 f

KeyboardInterrupt: 

In [32]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import optuna

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start timing the entire process
start_time = time.time()

# Assuming df is your initial DataFrame
# Split data
X = df.drop('Y', axis=1)
y = df['Y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.FloatTensor(y_train.values).unsqueeze(1).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.FloatTensor(y_test.values).unsqueeze(1).to(device)

class VIME(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(VIME, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        self.mask_predictor = nn.Linear(hidden_dim, input_dim)
        self.feature_predictor = nn.Linear(hidden_dim, input_dim)
        self.predictor = nn.Linear(hidden_dim, 1)

    def forward(self, x, mask=None):
        if mask is None:
            mask = torch.bernoulli(torch.ones_like(x) * 0.8)
        x_masked = x * mask
        h = self.encoder(x_masked)
        mask_pred = torch.sigmoid(self.mask_predictor(h))
        feature_pred = self.feature_predictor(h)
        y_pred = self.predictor(h)
        return y_pred, mask_pred, feature_pred

def vime_loss(y_true, y_pred, mask_true, mask_pred, feature_true, feature_pred, alpha=1.0, beta=1.0):
    prediction_loss = nn.MSELoss()(y_pred, y_true)
    mask_loss = nn.BCELoss()(mask_pred, mask_true)
    feature_loss = nn.MSELoss()(feature_pred, feature_true)
    return prediction_loss + alpha * mask_loss + beta * feature_loss

def objective(trial):
    # Define hyperparameters to tune
    hidden_dim = trial.suggest_int('hidden_dim', 32, 256)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    num_epochs = trial.suggest_int('num_epochs', 10, 100)
    alpha = trial.suggest_loguniform('alpha', 0.1, 10.0)
    beta = trial.suggest_loguniform('beta', 0.1, 10.0)

    # Create the VIME model
    model = VIME(X_train.shape[1], hidden_dim).to(device)

    # Define optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            mask = torch.bernoulli(torch.ones_like(batch_X) * 0.8)
            y_pred, mask_pred, feature_pred = model(batch_X, mask)
            loss = vime_loss(batch_y, y_pred, mask, mask_pred, batch_X, feature_pred, alpha, beta)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        y_pred, _, _ = model(X_test_tensor)
        mse = mean_squared_error(y_test_tensor.cpu().numpy(), y_pred.cpu().numpy())
        rmse = np.sqrt(mse)

    return rmse

# Perform hyperparameter tuning with Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params

# Train the final VIME model with the best hyperparameters
best_model = VIME(X_train.shape[1], best_params['hidden_dim']).to(device)
optimizer = optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

training_start_time = time.time()
for epoch in range(best_params['num_epochs']):
    best_model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        mask = torch.bernoulli(torch.ones_like(batch_X) * 0.8)
        y_pred, mask_pred, feature_pred = best_model(batch_X, mask)
        loss = vime_loss(batch_y, y_pred, mask, mask_pred, batch_X, feature_pred, best_params['alpha'], best_params['beta'])
        loss.backward()
        optimizer.step()

training_time = time.time() - training_start_time

# Evaluation
best_model.eval()
with torch.no_grad():
    inference_start_time = time.time()
    y_pred, _, _ = best_model(X_test_tensor)
    inference_time = time.time() - inference_start_time

    # Convert to numpy for metric calculation
    y_true = y_test_tensor.cpu().numpy()
    y_pred = y_pred.cpu().numpy()
    
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

# Calculate total computation time
computation_time = time.time() - start_time

# Store results in the existing result DataFrame
result.loc['VIME'] = [rmse, r2, training_time, inference_time, computation_time, best_params]

print(result)

# Print the best hyperparameters
print("\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")


[I 2024-07-26 08:36:20,514] A new study created in memory with name: no-name-42b9754c-a810-47f1-ad68-e644121d8d81


Using device: cpu


C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\884279414.py:69: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\884279414.py:72: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform('alpha', 0.1, 10.0)
C:\Users\Tommy\AppData\Local\Temp\ipykernel_6972\884279414.py:73: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  beta = trial.suggest_loguniform('beta', 0.1, 10.0)
[I 2024

                         RMSE R-squared Training Time Inference Time  \
Linear Regression    0.211628  0.999992      0.001003            0.0   
Ridge Regression     0.905454  0.999858      0.000999            0.0   
Lasso Regression     0.383952  0.999974           0.0            0.0   
ElasticNet           4.404609  0.996642      0.001995            0.0   
Decision Tree       36.807797    0.7655      0.000998            0.0   
Random Forest       37.345571  0.758598      0.391953       0.023935   
Gradient Boosting   32.551013  0.816603      0.060839       0.000996   
XGBoost             47.556448  0.608545      0.066822       0.000997   
LightGBM           105.104923 -0.912095      0.005985       0.000997   
CatBoost             61.82527    0.3384       0.16257       0.001988   
MLP                  7.094997  0.991287      0.594412       0.000998   
DNN                109.746297 -1.084697      0.063799            0.0   
DCN                   4.72361  0.996138      0.418879           